# Slideshow Project

This complicated project uses all the major components of the other projects to create a system that will display images and show information about the images all on the electronics connected to the Minnowboard.

### Review the wiki page at http://wiki.minnowboard.org/Projects/Maker_Slideshow for hardware requirements and setup.

Start by importing all the necessary packages.

In [ ]:
import time
import sys
import os
from PIL import Image
from pyDrivers.ada_lcd import *
import pyDrivers.ILI9341 as TFT
import Adafruit_GPIO as GPIO
import Adafruit_GPIO.SPI as SPI

Now we'll start by invoking the GPIO class, which will identify our board and initialize the pins. We will use two pins for input for scrolling through the slideshow. We default to the spidev device at <code>/dev/spidev0.0</code> for the minnow

Additionally, the Data/Command and Reset pins are defined for the TFT LCD display.

In [ ]:
myGPIO = GPIO.get_platform_gpio()

myGPIO.setup(12,GPIO.IN)
myGPIO.setup(16,GPIO.IN)

lcd = ADA_LCD()
lcd.clear()

SPI_PORT = 0
SPI_DEVICE = 0
SPEED = 16000000
DC = 10
RST = 14 


The following functions collect all the images in the specified directory and place them into a list. It will filter out all the non-image files in the directory. It will fail if no images are found.

In [ ]:
imageList = []
rawList = os.listdir("/notebooks")
for i in range(0,len(rawList)):
    if (rawList[i].lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))==True):
        imageList.append("/notebooks" + "/" + rawList[i])
    
if len(imageList)==0:
    print "No images found!"
    exit(1)
    
count = 0

print imageList

Now we'll initialize the TFT LCD display and clear it.

In [ ]:
disp = TFT.ILI9341(DC, rst=RST, spi=SPI.SpiDev(SPI_PORT,SPI_DEVICE,SPEED))
disp.begin()

This long infinite loop will work like so:

<b>Clear the char LCD, write name of new image</b>

<b>Wait for a button press</b>

<b>Try to open an image</b>

<b>Display the image on the TFT LCD</b>

--If we fail to open the file, print an error message to the LCD display--

----If we failed, open up the next file in the list. If we're at the end, restart at the beginning ----

In [ ]:
while True:
    
    lcd.clear()
    time.sleep(0.25)
    message = " Image " + str(count+1) + " of " + str(len(imageList)) + "\n" + imageList[count][len(sys.argv[1]):]
    lcd.message(message)
    lcd.scroll()
    try:
        image = Image.open(imageList[count])
    except(IOError):
        lcd.clear()
        time.sleep(0.25)
        message = " ERR: " + str(count+1) + " of " + str(len(imageList)) + "\n" + imageList[count][len(sys.argv[1]):]
        lcd.scroll()
        lcd.message(message)
        if(count == len(imageList)-1):
            image = Image.open(imageList[0])
        else:
            image = Image.open(imageList[count+1])

    image = image.rotate(90).resize((240, 320))
    disp.display(image)
    
    try:
        while True:
            if (myGPIO.input(12) != 1 and count != 0):
                count = count - 1
                break
            if (myGPIO.input(16) != 1 and count != len(imageList)-1):
                count = count + 1   
                break 
    except (KeyboardInterrupt):
        lcd.clear()
        lcd.message("Terminated")
        print
        exit(0)
